# ニューラルネットワークの訓練: 重みの更新回数と出力値

In [ ]:
import numpy as np
import PIL.Image as  Image
import time

import matplotlib.pyplot as plt

import torch as t
import torch.optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv

各種関数を定義

In [ ]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
device

In [ ]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(784, 1000) 
        self.l2 = nn.Linear(1000, 1000)
        self.l3 = nn.Linear(1000, 10)

    def forward(self, x):
        h = x.view(-1, 28*28) # (N, 1, 28, 28) -> (N, 784)
        # １層目
        h = F.relu(self.l1(h))
        # ２層目
        h = F.relu(self.l2(h))
        # 出力層
        return self.l3(h)

In [ ]:
def dev_env(tensor):
      return "cuda" if tensor.is_cuda else "cpu"

In [ ]:
def learn(model, batchsize, epochs):
    """
    指定されたモデルを指定回数分だけ訓練させる関数。
    """
    train_loader = t.utils.data.DataLoader(trainset, batch_size=batchsize)
    optimizer = t.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(device)
    model.train()
    model = model.to(device)
    for epoch in range(epochs):
        running_loss = 0.0
        start = time.time()
        for i, (inputs, labels) in enumerate(train_loader):
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            if False and i % 100 == 99:
                end = time.time()
                print('[{:d}, {:5d}] loss: {:.3f} (elapsed: {:.3f} [s] by {})'
                         .format(epoch + 1, i + 1, running_loss / 100, end-start, dev_env(outputs)))
                running_loss = 0.0
                start = time.time()
    return model

In [ ]:
def predict(model, test_data):
      model = model.eval()
      input_tensor = test_data[0]
      input_tensor.unsqueeze_(0)
      prediction = model(Variable(input_tensor).to(device))
      prediction.data[0] = F.softmax(prediction.data[0], dim=0)
      return prediction

In [ ]:
def print_prediction(prediction, test_data):
      prediction = prediction.cpu()
      data_volume = len(prediction.data[0])
      fig, ax = plt.subplots(1,2, figsize=(10, 5))
      # 確率グラフを表示
      ax[0].set_title("Prediction")
      ax[0].set_xticks(np.arange(0, data_volume, 1))
      ax[0].bar(x=range(data_volume), height=prediction.data[0])
      # 画像データを表示
      ax[1].set_title("Image")
      ax[1].imshow(test_data[0].reshape(28,28))

In [ ]:
def train_and_eval(model, test_index, batchsize=100, epoch=20):
    # Training process
    learned_model = learn(model, batchsize, epoch)
    # Evaluation process
    test_data = testset[test_index]
    prediction = predict(learned_model, test_data)
    print_prediction(prediction, test_data)

データをロードして準備しましょう。

In [ ]:
 preprocess = tv.transforms.Compose([
                                  tv.transforms.ToTensor(),
                                  tv.transforms.Normalize((0.5, ), (0.5, ))])

In [ ]:
trainset = tv.datasets.MNIST('~/tmp/mnist', 
                                           train=True,
                                           download=True,
                                           transform=preprocess)

In [ ]:
testset = tv.datasets.MNIST('~/tmp/mnist',
                                          train=False,
                                          download=True,
                                          transform=preprocess)

## トレーニングを繰り返す

In [ ]:
mlp = MLP()

In [ ]:
batchsize=10

In [ ]:
train_and_eval(model=mlp, test_index=1, batchsize=batchsize, epoch=1)

In [ ]:
train_and_eval(model=mlp, test_index=1, batchsize=batchsize, epoch=5)

In [ ]:
train_and_eval(model=mlp, test_index=1, batchsize=batchsize, epoch=10)

In [ ]:
train_and_eval(model=mlp, test_index=1, batchsize=batchsize, epoch=20)

In [ ]:
train_and_eval(model=mlp, test_index=1, batchsize=batchsize, epoch=50)

In [ ]:
train_and_eval(model=mlp, test_index=1, batchsize=batchsize, epoch=100)

In [ ]:
train_and_eval(model=mlp, test_index=1, batchsize=batchsize, epoch=100)

# sample_id=2

In [ ]:
train_and_eval(model=mlp, test_index=2, batchsize=batchsize, epoch=1)

In [ ]:
train_and_eval(model=mlp, test_index=2, batchsize=batchsize, epoch=5)

In [ ]:
train_and_eval(model=mlp, test_index=2, batchsize=batchsize, epoch=10)

In [ ]:
train_and_eval(model=mlp, test_index=2, batchsize=batchsize, epoch=50)

In [ ]:
train_and_eval(model=mlp, test_index=2, batchsize=batchsize, epoch=100)

In [ ]:
train_and_eval(model=mlp, test_index=2, batchsize=batchsize, epoch=200)

## ここで確認できること

ニューラルネットワークの訓練が進むにつれて、手書き文字列画像を入力すると、その手書き文字列が所属するクラス
の値が最大値を示すようになります。